# Introduction to simple Vectorization Methods

This is a simple introduction to some basic vectorization methods; namely Bag of Words, Count Vectorizer, Tfidf Vectorizer, Word2Vec, FastText, and Gensim. I will include a brief background on what each method does, notable strengths / weaknesses, and what the specific purposes of the vectorization method were. There will be a brief discussion on the benefits of BERT relative to these, but there will be no code for it.

## Basic Terminology:

- **Vectorizer**: A vectorization method is a mathematical formula or algorithm which transforms input text into a numerical vector. A vectorizer is the object that performs the vectorization on the input text. 


- **Document Term Matrix**: An D by V matrix, where there were C documents fed in, and a vocabulary of V different words. Cell (d, v) corresponds to the value of term v in document d. Usually refered to as **dtm**.


- **Syntactic Similarity**: Two words are syntatically similar if they share the same root word. Eg Exercise, Exercises, Exercised, and Exercising are all syntactically similar. Named Entity's which are based on real words are still considered syntactically similar, eg Apple (company) and apple (food). Homonyms may be syntactically similar, but are not always considered to be so, eg pen (writing instrument) and pen (animal holder) are similar, but not sea (body of water) and see (to be able to visualize something). 


- **Symantic Similarity**: Two words are symantically similar if they share the same meaning, ie synonyms. Similar meaning is a somewhat arbitrary definition, as how close the meaning of two words are 


- **Lemmatization**: Lemmatization is a text preprocessing method that typically occurs prior to text being fed into a vectorizer (though this depends on the intended purpose of the output vectors). It is a rules based method that attempts to normalize vocabulary; eg all words ending in -ing will have the -ing deleted. Similarily for ending in s (to capture plurals). This means "ending" would be replaced with "end", allowing the number of terms to be reduced. Lemmatization compares to stemming in that it is typically faster, but more coarse grained. 


- **Stemming**: Stemming is a text preprocessing method that typically occurs prior to text being fed into a vectorizer (though this depends on the intended purpose of the output vectors). It reduces words to their stem, but does not use a rules based method. Eg "ending" would be reduced to "end", but also "was" reducing to "be". It tends to cover edge cases better than lemmaticaztion, but is slower. 


- **Pretrained Vectors**: Pretrained Vectors are vectors, such as fastText vectors, which were trained on a huge corpus of text by some group (eg Facebook trained FastText vectors and made the vectors public to anyone) and are then available to be downloaded and used by an individual. Pretrained vectors usually produce better results when compared to small corpus sizes, but are typically not tuned to the specific vocabulary in the given corpus. Eg in military contexts / corpuses, the acronym "KM" may refer to knowledge management, but this will not be well trained in a pretrained corpus. Pretrained vectors are typically imported, then refined on the corpus in the given context.

In [6]:
import pprint as pprint
import numpy as np
from gensim.models import FastText
from gensim.models import Word2Vec
from nltk.tokenize import sent_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [3]:
doc1 = 'This is the first document. It is multiple sentances.'
doc2 = 'And this is the second document'
doc3 = 'Here is the third document. It is also multiple sentances.'
doc4 = 'Is this the first document'
corpus = [doc1, doc2, doc3, doc4]

## Bag Of Words

Bag of Words is the simplest vectorization method, and is closely related to Count Vectorizers. A bag of words model indicates what words show up in the given text, via one-hot-encoding. This means every single value in the document-terms matrix will always have a value of 1 or 0. 

### Pros and Cons:

#### Pros: 
- Features are always in the same range, and won't require scaling. 
- Simple, fast, and easily explainable.

#### Cons:
- Does not account for symantic similarity of vocabulary.
- Does not take into account context of words.
- Does not take into account order of terms / historical information.
- Cannot be used below the paragraph level of sample sizes (Eg cannot compare two words, only two texts). 
- Tends to be sparse, eg usually needs additional feature selection steps.

### Purpose: 
Bag of words is good for creating an initial baseline for comparison at paragraph+ level tasks. It is primarily useful for indicating whether words showed up at all in a given text, not indicating anything about the importance of the words within the text.

In [7]:
bowVec = CountVectorizer(binary=True) #bowVec is a vectorizer
dtm = bowVec.fit_transform(corpus)

In [12]:
bowVec.get_feature_names() #Use after calling fit_transform or fit

['also',
 'and',
 'document',
 'first',
 'here',
 'is',
 'it',
 'multiple',
 'second',
 'sentances',
 'the',
 'third',
 'this']

Here we see the vocabulary. Note that the vocabulary does not necessarily appear in the order from the text. 

In [13]:
dtm.toarray() 

array([[0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1],
       [0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1],
       [1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0],
       [0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1]], dtype=int64)

Notice that all items are indeed one or zero. Based on the get_feature_names(), we know 'also' is the first term in the vocabulary. It does indeed show up in only the third document. 

## Count Vectorizer 

Count Vectorizers are very similar to Bag Of Words, except instead of indicating whether or not a word showed up in a document, they indicate how many times a word occured. Eg dtm item (d, v) can be any positive integer.

### Pros and Cons:

#### Pros: 
- Simple, fast, and easily explainable.
- Some context on relative importance of words within the given text. 

#### Cons:
- Feature scaling may be required. 
- Does not account for symantic similarity of vocabulary.
- Does not take into account context of sub-pieces of text (eg sentences within a paragraph have no impact).
- A text's vector does not take into account context of other texts in the corpus.
- Does not take into account order of terms / historical information.
- Cannot be used below the paragraph level of sample sizes (Eg cannot compare two words, only two texts). 
- Tends to be sparse, eg usually needs additional feature selection steps.
- Cannot adapt to new vocabulary. 
- Feature vectors get very large with large input vocabularies.

### Purpose: 
Count Vectorizers are a good, easy to understand yet not totally trivial starting point. Also notably, the very common LDA topic modelling algorithm is designed to be built off of count vectorizer inputs (it assumes explicit counts of documents for its inputs). Many topic modelling algorithms are varients of LDA and may make a similar assumption.

In [16]:
cvVec = CountVectorizer(binary=False)
dtm = cvVec.fit_transform(corpus)

In [15]:
bowVec.get_feature_names()

['also',
 'and',
 'document',
 'first',
 'here',
 'is',
 'it',
 'multiple',
 'second',
 'sentances',
 'the',
 'third',
 'this']

In [17]:
dtm.toarray() 

array([[0, 0, 1, 1, 0, 2, 1, 1, 0, 1, 1, 0, 1],
       [0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1],
       [1, 0, 1, 0, 1, 2, 1, 1, 0, 1, 1, 1, 0],
       [0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1]], dtype=int64)

## Tfidf Vectorizer 

Tfidf Vectorizers are similar to Count Vectorizers, except instead of counting the occurances of words in the text, they indicate modified frequenies of words. Tf-idf stands for "Term Frequency - Inter-Document Frequency." Pure TF is sometimes used, but TFIDF is about incorporating how rare a term is within a given document as well as within the given corpus. 

The TF portion of the algorithm weights terms higher the more they appear in a document. So in a corpus of military documents, if a particular article is about tanks, the words "military" and "tank" would get a high score from the TF portion. 

The IDF portion of the algorithm reduces a vocabulary term's weight by the number of documents it shows up in. So if the entire corpus is military documents, but only a few are about tanks, the idf score of "tank" would be high, but "military" would be low. 

Combinging the TF-IDF portions of the algorithm, you get values for vocabulary features which help distinguish the words that are heavily occuring in only a small portion of documents. One can think of this as the algorithm including some basic indication of relative feature importance. The tfidf values within the DTM are all floats in the range of 0-1. 

### Pros and Cons:

#### Pros: 
- Good corpus level information capture
- Captures relative importance of words within documents
- Output is directly useful for other tasks such as keyword evaluation 

#### Cons:
- Cannot handle Out of Vocab Words well
- Cannot handle Symantic and syntactic similarity
- Sparse in large datasets 
- Doesn't account for order of terms

### Purpose: 
EDIT. Keywords Trivial. Identify important vocab words. Strong doc-doc comparison; excluding context.

In [18]:
tfidfVec = TfidfVectorizer()
dtm = tfidfVec.fit_transform(corpus)

In [24]:
pprint.pprint( tfidfVec.get_feature_names() )

['also',
 'and',
 'document',
 'first',
 'here',
 'is',
 'it',
 'multiple',
 'second',
 'sentances',
 'the',
 'third',
 'this']


In [23]:
pprint.pprint( dtm.toarray() )

array([[0.        , 0.        , 0.24524499, 0.37052242, 0.        ,
        0.49048999, 0.37052242, 0.37052242, 0.        , 0.37052242,
        0.24524499, 0.        , 0.29996983],
       [0.        , 0.55690079, 0.29061394, 0.        , 0.        ,
        0.29061394, 0.        , 0.        , 0.55690079, 0.        ,
        0.29061394, 0.        , 0.35546256],
       [0.39227182, 0.        , 0.20470371, 0.        , 0.39227182,
        0.40940742, 0.30927161, 0.30927161, 0.        , 0.30927161,
        0.20470371, 0.39227182, 0.        ],
       [0.        , 0.        , 0.38408524, 0.58028582, 0.        ,
        0.38408524, 0.        , 0.        , 0.        , 0.        ,
        0.38408524, 0.        , 0.46979139]])


Note that indeed all values are floats. Further, note that because of our corpus size, certain words have the exact same frequencies or interdocument frequencies. Note that these float values are all significantly higher than would be seen in normal TFIDF matrices simply due to the tiny size of our corpus.

## Word2Vec

Word2Vec is significantly different from our previous models, and is much more similar to FastText. Word2Vec is whats known as an embedding model. In this model, each word has its own individual vector representation learned. So unlike our previous models, Word2Vec creates vectors for individual words rather than entire documents (there is a similar algorithm called Doc2Vec which acts as a scaled up version for documents, but word2vec is easier to explain). 

Word vectors are created by looking at the surrounding context during vector training, namely what other words appear in the immediate vicinity of the trained word. Typically local context is limited to the current sentence (thus sentance tokenization is an important subtask for training Word2Vec). The context words inform the weights of each vector. Since words used in the same context generally have the same meaning, word2vec (and other embedding models) will be much better at capturing the relative meaning and similarity between different words. 

When training the actual vectors, the Word2Vec needs to know how many dimensions one wants to embed words within. In most contexts, this value is typically 75, 150, or 300. Each of these dimensions will be a float value from -1 to 1. These dimensions to NOT correspond *directly* to anything specific; eg one dimension will not be indicative of past/present tense. However, it is possible to look at correlations between dimensions and certain word features such as plurality, tense, etc.   

### Pros and Cons:

#### Pros: 
- Capable of Symantic Similarity
- Easily handles syntactic similiarity
- Dense, short feature vectors (minimal additional processing necessary)

#### Cons:
- No ability to handle out-of-vocab words
- Poor handling of document level tasks / does not contextualize document level information
- Does not have context dependant embeddings, the embedding for a given word is always the same

### Purpose: 
Word2Vec vectors are word specific, and do not directly take into account anything about the overall corpus structure. This is why they tend to perform word on document-level tasks. However, the algorithm's ability to capture symantic similarity and word relatedations are also direct results of training using individual words. 

##### Special Note: Its important that w2v is fed in a list of lists; the main list is a list of sentences, the sub list is a list of words in each sentence. So both sentance and word tokenization is important

In [39]:
corpus_sentences = np.array( [sent_tokenize(text) for text in corpus], dtype=object )
sentences = np.hstack( corpus_sentences )
tokenized_sentences = [sentence.split() for sentence in sentences]

model = Word2Vec( tokenized_sentences, vector_size=5, window=3, min_count=1, workers=10)
vocab = list(model.wv.index_to_key)

print( tokenized_sentences ) 
print()

for word in vocab:
    print( model.wv[word], ' | ', word )

[['This', 'is', 'the', 'first', 'document.'], ['It', 'is', 'multiple', 'sentances.'], ['And', 'this', 'is', 'the', 'second', 'document'], ['Here', 'is', 'the', 'third', 'document.'], ['It', 'is', 'also', 'multiple', 'sentances.'], ['Is', 'this', 'the', 'first', 'document']]

[-0.01072454  0.0047286   0.10206699  0.18018547 -0.186059  ]  |  is
[-0.14236456  0.12920347  0.17945339 -0.10033613 -0.07524883]  |  the
[ 0.14756316 -0.03061825 -0.09073066  0.13099015 -0.09716446]  |  document
[-0.03634403  0.0575913   0.01982498 -0.16581543 -0.1889571 ]  |  this
[0.14623532 0.10140524 0.13515386 0.01525731 0.12701778]  |  sentances.
[-0.06814497 -0.01890523  0.11535613 -0.1504877  -0.07872236]  |  multiple
[-0.15027198 -0.01862813  0.19080846 -0.1463472  -0.04669569]  |  It
[-0.03875484  0.1615487  -0.11861791  0.00090322 -0.0950747 ]  |  document.
[-0.192071    0.10014586 -0.17519173 -0.0878365  -0.000702  ]  |  first
[-0.00592365 -0.1532248   0.19229484  0.09964113  0.18466286]  |  Is
[-0.16

Also note that it is easy to compare words. Due to the absolutely tiny size of our corpus, words distances and similarities aren't quite as accurate as they would be in a full corpus. Normally "this" and "This" would have a similarity value of almost 1. 

In [36]:
model.wv.most_similar('This')[0]

('this', 0.8035771250724792)

## FastText

FastText, like Word2Vec, is an embedding model that looks at nearby tokens to learn how to best represent the token currently being trained. However, unlike word2vec, fastText is designed to embed at the character ngram level. This means that a word such as "this", if broken up into chunks of size 2+, will be split into "th", "hi", "is", "thi", "his" and "this". Each chunk of characters is learned, and then the understanding of each sub-piece of a word is combined to create the overall vector for each word. This means that many different parts of language are learned, eg pluralality is captured, as is common endings such as "ing". 

When training the actual vectors, FastText also needs to know how many dimensions one wants to embed words within. In most contexts, this value is typically 75, 150, or 300. Each of these dimensions will be a float value from -1 to 1. These dimensions to NOT correspond *directly* to anything specific; eg one dimension will not be indicative of past/present tense. However, it is possible to look at correlations between dimensions and certain word features such as plurality, tense, etc. FastText is able to capture the sub-word level features, which leads towards good syntactic similarity use cases.

### Pros and Cons:

#### Pros: 
- Capable of Symantic Similarity
- Does Not need Preprocessing
- Handles Out of Vocab Words
- Dense, short feature vectors (minimal additional processing necessary)

#### Cons:
- Poor handling of document level tasks / does not contextualize document level information
- Does not have context dependant embeddings, the embedding for a given word is always the same

### Purpose: 
FastText vectors are word specific, and do not directly take into account anything about the overall corpus structure. This is why they tend to perform word on document-level tasks. However, the algorithm's is quite good at capturing both symantic and syntactic similarity in addition to word relatedations. Its simple (minimal preprocessing necessary) as well as fast, so is typically a good starting point for these sorts of tasks.  